In [1]:
import numpy as np 
import pandas as pd 

import sys
sys.path.append("/scratch/m/murray/dtolgay")

from get_spaxels import find_indices_inside_boxes

from tools import functions_readfiles as readfiles
from tools import constants
from tools import functions_importing_observations as observations

import h5py

import matplotlib
import matplotlib.pyplot as plt

# Reading CLOUDY data

In [2]:
# Read the cloudy particles 

base_dir = "/scratch/m/murray/dtolgay/post_processing_fire_outputs/skirt/runs_hden_radius"

# galaxy_name = "gal0"
# galaxy_type = "firebox"

galaxy_name = "m12i_res7100_md"
galaxy_type = "zoom_in"

# directory_name = "voronoi_1e5_delete" # TODO
# directory_name = "seperated_firebox_galaxies"
directory_name = "voronoi_1e5"

file_name = "abundance_RBFInterpolator_smoothingLength.txt"


cloudy_runs = {
    "z0": {
        "redshift": "0.0",
        "data": pd.DataFrame(),
        "color": "black",
    },
    "z1": {
        "redshift": "1.0",
        "data": pd.DataFrame(),
        "color": "red",
    },
    "z2": {
        "redshift": "2.0",
        "data": pd.DataFrame(),
        "color": "orange",
    },
    "z3": {
        "redshift": "3.0",
        "data": pd.DataFrame(),
        "color": "green",
    },    
}


for run in cloudy_runs.keys():
    # Read the gas particles 
    redshift = cloudy_runs[run]["redshift"]
    fdir = f'{base_dir}/{galaxy_type}/z{redshift}/{galaxy_name}/{directory_name}/{file_name}'
    try:
        gas_particles, file_specific_columns = readfiles.read_interpolated_files_usingFilePath(path = fdir, interpolation_type="abundance")
    except:
        gas_particles, file_specific_columns = readfiles.read_interpolated_files_usingFilePath2(path = fdir, interpolation_type="abundance")

    # Save the data
    gas_particles['mass_co'] = gas_particles['mass']/gas_particles['mu_theoretical'] * gas_particles['fCO'] * constants.M_sun2gr # Msolar
    gas_particles['mass_h2'] = gas_particles['mass']/gas_particles['mu_theoretical'] * gas_particles['fh2'] * constants.M_sun2gr # Msolar
    cloudy_runs[run]["data"] = gas_particles


# Main

In [5]:
def calculate_spaxels(gas:pd.DataFrame, condition:dict, resolution=1e3) -> pd.DataFrame:

    """
    resolution: pc
    """

    # Calculate the spaxel coordinates
    indices, lower_left_corner_coordinates, upper_right_corner_coordinates, box_coordinates = find_indices_inside_boxes(
        max_length = condition["Rgal"], 
        resolution = resolution, 
        particles_df = gas, 
        overlap_shift=0
    )


    values_for_all_spaxels = []

    for indices_in_cell, lower_left_corner_coordinate in zip(indices, lower_left_corner_coordinates):
        
        # Initiate a dictionary to store the calculated values 
        calculated_properties = {} # Fresh start in each iteration.
        
        # Calculate the center coordinates of the spaxel
        x_center = lower_left_corner_coordinate[0] + resolution/2   # pc
        y_center = lower_left_corner_coordinate[1] + resolution/2   # pc     
        distance = np.sqrt(x_center**2 + y_center**2)               # pc 
        
        
        calculated_properties.update({
            "x_center": {
                "unit": "pc",
                "value": x_center,
                "column_name": "x_center",
            },
            "y_center": {
                "unit": "pc",
                "value": y_center,
                "column_name": "y_center",
            },
            "distance": {
                "unit": "pc",
                "value": distance,
                "column_name": "distance",
            },                        
        })    
        
        
        # Define the area
        area = resolution * resolution * constants.pc2cm**2 # cm2

        # Filter the particles 
        filtered_gas = gas.iloc[indices_in_cell].copy()
        
        # Calculate the total mass 
        total_co_mass = sum(filtered_gas["mass_co"])
        total_h2_mass = sum(filtered_gas["mass_h2"])
        

        # Calculate the surface number density 
        Nco = total_co_mass / area / constants.mco_gr # cm-2
        Nh2 = total_h2_mass / area / constants.mh2_gr # cm-2
        Nh = sum(filtered_gas["mass"]) * constants.M_sun2gr / area / constants.mh2_gr/2 # cm-2

        # Calculate average gas metallicity 
        mass_averaged_gas_metallicity = sum(filtered_gas["mass"] * filtered_gas["metallicity"]) / sum(filtered_gas["mass"]) # Zsolar

        # # Calculate Lco 
        # L_co_10 = sum(filtered_gas["mass_co"] * filtered_gas["L_co_10"]) / sum(filtered_gas["mass_co"]) / area # K km/s  # TODO
        
        calculated_properties.update({
            "Nco": {
                "unit": "cm-2",
                "value": Nco,
                "column_name": "Nco",
            },
            "Nh2": {
                "unit": "cm-2",
                "value": Nh2,
                "column_name": "Nh2",
            },
            "metallicity": {
                "unit": "Zsolar",
                "value": mass_averaged_gas_metallicity,
                "column_name": "metallicity",
            },
            "Nh" : {
                "unit": "cm-2",
                "value": Nh,
                "column_name": "Nh",
            },
            # "L_co_10": {
            #     "unit": "K km/s",
            #     "value": L_co_10,
            #     "column_name": "L_co_10",
            # } # TODO
        })
        
        
        # I calculated the spaxel properties. Now I will add it into the dataframe
        # Start a new arrays in after each spaxel.        
        values_for_this_run = []     
        column_names = [] 
        units = []   
        for key in list(calculated_properties.keys()):
            values_for_this_run.append(calculated_properties[key]["value"])
            column_names.append(f'{calculated_properties[key]["column_name"]}')
            units.append(f'{calculated_properties[key]["unit"]}')

        values_for_all_spaxels.append(values_for_this_run)



    ##### Outside of the loop 
    values_for_all_spaxels = np.array(values_for_all_spaxels)
    data_df = pd.DataFrame(values_for_all_spaxels, columns=column_names)        

    return data_df



In [6]:
resolution = 5e2

# Use only the cloudy particles within certain condition 
conditions = {
    "5kpc+disk":{
        "z_max": 500, # pc
        "Rgal": 5e3,  # pc
    }
}

for run in cloudy_runs.keys():
    cloudy = cloudy_runs[run]["data"]
    spaxels_cloudy = calculate_spaxels(gas=cloudy, condition=conditions["5kpc+disk"], resolution=resolution)
    cloudy_runs[run]["spaxels"] = spaxels_cloudy



1 finished. Left 399
Number of boxes: 400
Done!


KeyError: 'L_co_10'

# Import observations

In [ ]:
rachford = observations.rachford_2002_number_column_density(filedir = "/home/m/murray/dtolgay/Observations")

In [ ]:
for run in cloudy_runs.keys():
    spaxels_cloudy = cloudy_runs[run]["spaxels"]
    label = f"CLOUDY -- {cloudy_runs[run]['redshift']} -- resolution:{resolution} pc"
    plt.scatter(
        np.log10(spaxels_cloudy["Nh2"]),
        np.log10(spaxels_cloudy["Nco"]),
        label = label,
        color = cloudy_runs[run]["color"],
        s = 10,
    )

plt.scatter(
    rachford["logN(H2)"],
    rachford["log(N(CO))"],
    label = "Rachford et. al 2002"
)


plt.xlim(20,22)
plt.ylim(14, 18)

plt.legend()
plt.grid(True)
plt.ylabel("log(Nco) [cm-2]")
plt.xlabel("log(Nh2) [cm-2]")
plt.show()

In [ ]:
def bin_data(spaxel_data):

    # Take the log of the data and bin log values 
    spaxel_data = spaxel_data.copy()
    spaxel_data["log_Nh2"] = np.log10(spaxel_data["Nh2"])
    spaxel_data["log_Nco"] = np.log10(spaxel_data["Nco"])

    # Determine the bin centers 
    bins_Nh2 = np.arange(12, 22, 0.3)
    # Digitize the data into bins
    bin_indices = np.digitize(spaxel_data["log_Nh2"], bins_Nh2)

    # Create a list to store the binned data
    binned_data = []
    binnested_data = []
    # Iterate over the bins
    for i in range(len(bins_Nh2) - 1):
        # Select the data within the bin
        bin_data = spaxel_data[bin_indices == i + 1]
        
        # Calculate the mean and standard deviation of Nco in the bin
        if len(bin_data) > 0:
            mean_Nco = np.mean(bin_data["log_Nco"])
            std_Nco = np.std(bin_data["log_Nco"])
            binned_data.append([bins_Nh2[i], mean_Nco, std_Nco])

    # Create df
    binned_data = pd.DataFrame(binned_data, columns=["log_Nh2", "mean_log_Nco", "std_log_Nco"])

    return binned_data


# Run the code 
spaxels_cloudy = cloudy_runs['z0']["spaxels"]
bin_data(spaxel_data=spaxels_cloudy)

In [ ]:
plt.figure(figsize=(10, 6))


for run in cloudy_runs.keys():
    spaxels_cloudy = cloudy_runs[run]["spaxels"]
    binned_cloudy = bin_data(spaxel_data=spaxels_cloudy)
    label = rf"CLOUDY -- {cloudy_runs[run]['redshift']} -- 1 $\sigma$ average"
    plt.fill_between(
        x = binned_cloudy["log_Nh2"],
        y1 = binned_cloudy["mean_log_Nco"] - binned_cloudy["std_log_Nco"],
        y2 = binned_cloudy["mean_log_Nco"] + binned_cloudy["std_log_Nco"],
        label = label,
        color = cloudy_runs[run]["color"],
        alpha = 0.3,   
    )
    plt.plot(
        binned_cloudy["log_Nh2"],
        binned_cloudy["mean_log_Nco"],
        color = cloudy_runs[run]["color"],
        alpha = 1,   
    )

plt.scatter(
    rachford["logN(H2)"],
    rachford["log(N(CO))"],
    label = "Rachford et. al 2002"
)

plt.xlim(15,21.5)
# plt.ylim(12, 18)

plt.legend()
plt.grid(True)
plt.ylabel("log(Nco) [cm-2]")
plt.xlabel("log(Nh2) [cm-2]")
plt.show()